<a href="https://colab.research.google.com/github/lgeras/SPDV/blob/main/multiChain_SPDVs_CoM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Spatial Proximity to Disease Variants
---
<img src='https://drive.google.com/uc?id=1HrVTgv2bJdZWYZ7RRmkThc0uGudawcMT' title="Example SPDV derivation in the case of single closest disease position (SPDV_1)" width=500 align='right' hspace='20'/>



<p align = "justify">Most current variant effect predictors (VEPs) have been shown to be less effective at identifying non-loss-of-function (non-LOF) disease variants, which arise due to a gain of function (GOF) mechanism or manifest through the dominant-negative (DN) effect. Interestingly, disease mutation positions in non-LOF proteins have been shown to exhibit spatial clustering within protein structures. We have previously shown that <a href = https://colab.research.google.com/drive/11u6RuvZHZmyCXT8vavdrtFfW5rEbMEnG>Extent of Disease Clustering (EDC)</a>, a structure-derived gene-level metric, can be used to prioritize putative non-LOF genes. Analogously, we have demonstrated the utility of a structure-derived spatial distance metric at the position level, using Spatial Proximity to Disease Variants (SPDV) to score the pathogenicity of RyR1 receptor variants.</p>

<p align = "justify">In this work, we scale the two structure-informed metrics – EDC & SPDV – across thousands of human proteins with known disease variants by leveraging AlphaFold 2 models. We show that EDC effectively separates LOF and non-LOF disease genes, particularly when low-confidence disordered regions are excluded. In comparison with 72 dedicated variant effect predictors (VEPs), SPDV exceeds or is at least as effective as half of the tested models at identifying dominant disease variants. Interestingly, when exploring mechanism-specific performance, SPDV is equivalent to the top methods for GOF-associated disease. We show that SPDV performance consistently improves with increasing target EDC values, and we identify an EDC threshold after which SPDV consistently surpasses all other predictors, despite their superior general performance.</p>


<p align = "justify">Although the knowledge-based nature of SPDV limits its widespread utility as a VEP, its use as an analytical metric enables the identification of a subset of putative GOF disease genes where current VEP methods underperform relative to SPDV, indicating they may be improved by incorporating spatial distance information.</p>


We provide this notebook to allow easy calculations of SPDVs for all amino acid residue positions in an input structure, given a list of known disease variant positions.

In [ ]:
#@title Set up the notebook
#@markdown Install R dependencies and load packages. Needs to be run once (~2.5min).

!pip install -q rpy2
%load_ext rpy2.ipython

# Define and install R package list
%R required_packages <- c('dplyr', 'tidyr', 'stringr', 'bio3d', 'purrr','reticulate')
%R install_missing <- required_packages[!sapply(required_packages, require, character.only = TRUE)]
%R if (length(install_missing) > 0) install.packages(install_missing, quiet = TRUE)
%R sapply(required_packages, require, character.only = TRUE)


In [ ]:
#@title Upload files and settings { display-mode: "form" }

#@markdown ### <b><font color='greenyellow'>Upload a PDB structure and specify molecular context</font></b>
#@markdown Once the code cell is inititated, you will be prompted to upload a <b><font color='greenyellow'>**PDB structure (file extension .pdb)**</font></b> <ins>in the output zone of the cell</ins>. Multimeric structures and inter-chain SPDV calculations are supported.<br><br>
#@markdown <b><font color='greenyellow'>**Specify the structural chains**</font></b> to use for calculations (e.g `A` or `A,B,C`). Calculating SPDVs using multiple chains will require the provision of disease variant positions in a chain:position format, unless the input is a homomeric protein and the relevant option is selected.
chains_to_use = "A"  #@param {type:"string"}

#@markdown ### <b><font color='greenyellow'>Submission of disease variant positions</font></b>
#@markdown Known disease variant positions for single-chain runs are accepted as positions (`678, 687, 692`), wild-type residues (`D678, K687, A692`) or variants (`D678P, K687E, A692G`). <b><font color='greenyellow'>**Position numbering must correspond to the input structure**</font></b>. A wild-type residue check is performed if positions are supplied with wild-type residue identities.<br><br>
#@markdown In the case of a heteromeric multi-chain structure, chain identifiers (e.g `A:, B:`) must be provided for each position. For example, `A:678, B:687, C:692`, or `A:D678P, B:K687E, C:A692G`.<br><br>
#@markdown All positions can be provided in a <b><font color='greenyellow'>**tab-delimited (.txt or .tsv) file**</font></b>, with one input per line. If this option is selected you will be prompted to upload an additional file.

use_mutation_file = True  #@param {type:"boolean"}

#@markdown Alternatively, input positions or variants in a comma or space separated string:
manual_positions = "673,677,678,687,692,693,694,705,709,713,714,715,716,717,719"  #@param {type:"string", depends_on:"not use_mutation_file"}

#@markdown With this option enabled, a position-only input will be applied to all selected chains.
homomeric = False  #@param {type:"boolean"}

from google.colab import files
import re
import pandas as pd
from rpy2.robjects import StrVector, IntVector
import rpy2.robjects as robjects

# Upload PDB file
print("📂 Please select your PDB structure file (.pdb)")
uploaded = files.upload()
pdb_file = next((f for f in uploaded.keys() if f.lower().endswith(".pdb")), None)
print(f"✅ PDB file: {pdb_file}")

# Parse chain selection
selected_chains = [c.strip() for c in chains_to_use.upper().split(",") if c.strip()]

# Prepare mutation input
mutation_file = None
mutation_positions = []
mutation_chains = []
mutation_wts = []

if use_mutation_file:
    print("\n📂 Please select your mutation list file (.txt or .tsv, one mutation per line)")
    uploaded = files.upload()
    mutation_file = next((f for f in uploaded.keys() if f.lower().endswith(('.tsv', '.txt'))), None)
    print(f"✅ Mutation file: {mutation_file}")

    try:
        mut_df = pd.read_csv(mutation_file, sep="\t", header=None, dtype=str)
        raw_input = " ".join(mut_df.fillna("").astype(str).values.flatten())
    except Exception as e:
        print(f"\n⚠️ Failed to parse mutation file: {e}")
        raw_input = ""
else:
    print("\n📝 Using manual mutation input:")
    raw_input = manual_positions
    print(f"Raw input: {raw_input}")

# Split mutation entries using regex (handles whitespace and commas)
mutation_entries = re.split(r'[\s,]+', raw_input.strip())

# Parse mutation strings (formats: A:V34L, V34L, 34, A:34)
pattern = re.compile(r'(?:(?P<chain>[A-Za-z]):)?(?P<wt>[A-Za-z])?(?P<pos>\d+)(?:[A-Za-z])?')
seen = set()

for entry in mutation_entries:
    if not entry:
        continue
    match = pattern.fullmatch(entry)
    if not match:
        raise ValueError(f"❌ Failed to parse mutation input: {entry}")

    input_chain = match.group("chain")
    pos = int(match.group("pos"))
    wt = (match.group("wt") or "").upper()

    if input_chain:
        # Explicit chain provided
        key = (input_chain, pos)
        if key not in seen:
            seen.add(key)
            mutation_chains.append(input_chain)
            mutation_positions.append(pos)
            mutation_wts.append(wt)
    elif homomeric:
        # No chain given, replicate across all selected chains
        for ch in selected_chains:
            key = (ch, pos)
            if key not in seen:
                seen.add(key)
                mutation_chains.append(ch)
                mutation_positions.append(pos)
                mutation_wts.append(wt)
    else:
        # No chain and not homomeric — must have single chain selected
        if len(selected_chains) != 1:
            raise ValueError(f"❌ Chain info missing for '{entry}' and multiple chains selected.")
        ch = selected_chains[0]
        key = (ch, pos)
        if key not in seen:
            seen.add(key)
            mutation_chains.append(ch)
            mutation_positions.append(pos)
            mutation_wts.append(wt)

# Summary
print(f"\n✅ Parsed {len(mutation_positions)} unique mutation(s):")

def format_mutation_label(c, p, w):
    return f"{c}:{w}{p}" if w else f"{c}:{p}"

labels = [format_mutation_label(c, p, w) for c, p, w in zip(mutation_chains, mutation_positions, mutation_wts)]

# Limit to 20 shown items
max_display = 20
if len(labels) <= max_display:
    # Print in rows of 4
    for i in range(0, len(labels), 4):
        print("  " + "  ".join(labels[i:i+4]))
else:
    print("  Showing first 20 of", len(labels), "mutations:")
    for i in range(0, max_display, 4):
        print("  " + "  ".join(labels[i:i+4]))
    print("  ... (truncated)")


# Export to R
robjects.globalenv['pdb_file'] = pdb_file
robjects.globalenv['mutation_positions'] = IntVector(mutation_positions)
robjects.globalenv['mutation_chains'] = StrVector(mutation_chains)
robjects.globalenv['mutation_wts'] = StrVector(mutation_wts)
robjects.globalenv['selected_chains'] = StrVector(selected_chains)

# Distance settings
#@markdown ### <b><font color='greenyellow'>SPDV calculation options</font></b>
#@markdown Select the number of closest disease variant positions to take into account when calculating the SPDV. SPDV_1 represents the distance to the single closest position, while SPDV_3 is the mean distance to the 3 closest positions, respectively.
distances_to_average = 1  #@param {type:"slider", min:1, max:40, step:1}
#@markdown Alternatively, derive all SPDVs in the range 1:40.
run_all_averages = True  #@param {type:"boolean"}
#@markdown SPDVs are calculated between amino acid residue centers of mass. Deselect this option to derive distances between residue alpha carbon atoms.
use_CoM = True  #@param {type:"boolean"}

robjects.globalenv['k_nearest'] = distances_to_average
robjects.globalenv['run_all'] = run_all_averages
robjects.globalenv['use_com'] = robjects.BoolVector([use_CoM])[0]


In [ ]:
%%R
#@title Run & Save { display-mode: "form" }

#@markdown ### <b><font color='greenyellow'>Output options</font></b>

#Save?
True <- TRUE
False <- FALSE

#@markdown Would you like to download the output as a file?
save_spdvs <- True  #@param {type:"boolean"}

#@markdown Provide the output file name with a file extension.
output_file <- "SPDV_output.txt"  #@param {type:"string"}

# Read and filter PDB
pdb_obj <- read.pdb(file = pdb_file)
present_chains <- unique(pdb_obj$atom$chain)
missing_chains <- setdiff(selected_chains, present_chains)

if (length(missing_chains) > 0) {
  stop("❌ Chain(s) not found in structure: ", paste(missing_chains, collapse = ", "))
} else {
  message("✅ All requested chains are present in the structure: ", paste(selected_chains, collapse = ", "))
}

pdb_obj$atom <- pdb_obj$atom[pdb_obj$atom$chain %in% selected_chains, ]
calpha_atoms <- pdb_obj$atom[pdb_obj$calpha & pdb_obj$atom$chain %in% selected_chains, ]

# Mutation dataframe
Disease_pos <- tibble(
  chain = mutation_chains,
  resno = mutation_positions,
  wt = mutation_wts
)

# Coordinate reference: CoM or C-alpha
if (use_com) {
  message("✅ Using center of mass coordinates.")
  get_com <- function(res_no, chain_id) {
    sel <- atom.select(pdb_obj, resno = res_no, chain = chain_id)
    com_coords <- com.xyz(
      pdb_obj$xyz[, sel$xyz],
      mass = atom2mass(pdb_obj$atom[sel$atom, "elety"])
    )
    tibble(chain = chain_id, resno = res_no, x = com_coords[1], y = com_coords[2], z = com_coords[3])
  }
  disease_coms <- map2_dfr(unique(calpha_atoms$resno), unique(calpha_atoms$chain), get_com)
  temp_pdb <- inner_join(select(calpha_atoms, -x, -y, -z), disease_coms, by = c("resno", "chain"))
} else {
  message("⚠️ Using C-alpha coordinates.")
  temp_pdb <- calpha_atoms
}

temp_pdb = temp_pdb %>% filter(type == "ATOM")

# Match mutations
disease_temp <- inner_join(Disease_pos, temp_pdb, by = c("resno", "chain"))

# Optional WT check
if (any(Disease_pos$wt != "")) {
  wt_check <- inner_join(Disease_pos, temp_pdb, by = c("resno", "chain"))
  resid_one_letter <- aa321(wt_check$resid)
  mismatch_idx <- which(wt_check$wt != resid_one_letter)
  if (length(mismatch_idx) > 0) {
    bad <- wt_check[mismatch_idx, ]
    bad$resid_one <- resid_one_letter[mismatch_idx]
    message("❌ WT mismatch at positions:")
    print(bad[, c("chain", "resno", "wt", "resid", "resid_one")])
    stop("❌ WT mismatch between provided mutations and structure residues.")
  }
}



# Distance calculation
SPDVs <- map_dfr(seq_len(nrow(temp_pdb)), function(i) {
  this_res <- temp_pdb[i, ]
  zx <- this_res$resno
  ch <- this_res$chain
  is_disease <- any(disease_temp$resno == zx & disease_temp$chain == ch)
  Class <- ifelse(is_disease, "Disease", "Neutral")

  other_disease <- if (is_disease) {
    filter(disease_temp, !(resno == zx & chain == ch))
  } else {
    disease_temp
  }

  distances <- dist.xyz(this_res[, c("x", "y", "z")], other_disease[, c("x", "y", "z")])
  min_index <- which.min(distances)
  closest <- other_disease[min_index, ]

  base_info <- tibble(
    Chain = ch,
    Residue = zx,
    res_pLDDT = this_res$b,
    WT = aa321(this_res$resid),
    Position_type = Class,
    Proximal_DisPos = paste0(closest$chain, ":", closest$resno)
  )

  if (run_all) {
    dist_features <- setNames(lapply(1:40, function(k) mean(sort(distances)[1:k])), paste0("SPDV_", 1:40))
    bind_cols(base_info, as_tibble(dist_features))
  } else {
    base_info[[paste0("SPDV_", k_nearest)]] <- mean(sort(distances)[1:k_nearest])
    base_info
  }
})

# Save file
if (save_spdvs) {
  write.table(SPDVs, file = output_file, sep = "\t", row.names = FALSE, quote = FALSE)
  py_run_string(paste0("from google.colab import files; files.download('", output_file, "')"))
}

SPDVs
